### Importing libraries

In [1]:
import openravepy
import trajoptpy
import json
import numpy as np
import trajoptpy.kin_utils as ku
from trajoptpy.check_traj import traj_is_safe
import humanoidspy
import time
import random 
import matplotlib.pyplot as plt
import pickle
from trajopt_util import *
from planning_util import *
from regression import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

FILENAME = 'data/atlas_planning/'

### Setting the trajopt

In [2]:
np.set_printoptions(precision=4, suppress=True)

env = openravepy.RaveGetEnvironment(1)

In [3]:
if env is None:
    env = openravepy.Environment()
    env.StopSimulation()
    env.Load("../bigdata/atlas.xml")
    env.Load("../env/bookshelves.env.xml")
#iewer = trajoptpy.GetViewer(env)
env.SetViewer('qtcoin')

True

### Setting the robot

In [4]:
robot = env.GetRobots()[0]        
init_transform = np.eye(4)
init_transform[:3,3] = [-.35, 1, .92712]
init_transform[:3,3] = [-0.15, 0., .92712]
#init_transform[:3,3] = [2.6, 1, .92712]
robot.SetTransform(init_transform)

robot.SetDOFValues([-1.3],[robot.GetJoint("l_arm_shx").GetDOFIndex()])
robot.SetDOFValues([1.3],[robot.GetJoint("r_arm_shx").GetDOFIndex()])
robot.SetActiveDOFs(np.arange(robot.GetDOF()), openravepy.DOFAffine.Transform)

standing_posture = robot.GetActiveDOFValues()

dof = robot.GetActiveDOF()

In [5]:
def request_skeleton(n_steps):
    request = {
        "basic_info" : {
            "n_steps" : n_steps,
            "manip" : "active",
            "start_fixed" : True
        },
        "costs" : [
        {
            "type" : "joint_vel",            
            "params": {"coeffs":[1]}
        },
        {
            "type" : "joint_pos",
            "params" : {"coeffs": ([0.1]*28 + [0]*7), "vals":standing_posture.tolist()}
        },
        {
            "type":"collision",
            "params":{"coeffs":[2], "dist_pen":[.005], "continuous":False}
        },
        {
            "type":"collision","name":"cont_coll",
            "params":{"coeffs":[2], "dist_pen":[.01], "continuous":True}
        }
        ],
        "constraints" : [
        ],
        "init_info" : {
            "type" : "stationary"
        }
    }
    for i in xrange(1,n_steps):
        request["costs"].extend([
         #{
             #"type":"potential_energy",
             #"params":{"coeff" : .0005,"timestep":i}
         #},
        #{
            #"type":"static_torque",
            #"params":{"coeff" : .01,"timestep":i}
        #}                    
        ])
    return request  

def press_button_request(robot, hand_xyz, hand_link, foot_links, n_steps):
    """
    Sets up the problem to safely shift the weight to the other foot (to_foot)
    Suppose to_foot = "r_foot"    
    Then problem constrains both feet to remain at fixed poses (their current poses)
    at all intermediate timesteps, the center of mass lies over the convex hull of l_foot and r_foot
    at the final timestep, the center of mass lies over r_foot
    """    
    
    from_foot, to_foot = foot_links
    
    request = request_skeleton(n_steps)
    from_foot_xyz, from_foot_quat = xyzQuatFromMatrix(robot.GetLink(from_foot).GetTransform())
    to_foot_xyz, to_foot_quat = xyzQuatFromMatrix(robot.GetLink(to_foot).GetTransform())
    
    for i in xrange(1, n_steps):
        request["constraints"].extend([
            {
                "type":"pose",
                "name":"from_foot_pose",
                "params":{
                    "xyz":list(from_foot_xyz),
                    "wxyz":list(from_foot_quat),
                    "link":from_foot,
                    "timestep":i
                }
            },
            {
                "type":"pose",
                "name":"to_foot_pose",
                "params":{
                    "xyz":list(to_foot_xyz),
                    "wxyz":list(to_foot_quat),
                    "link":to_foot,
                    "timestep":i
                }
            }
        ])    
        request["constraints"].append(
            {
                "type":"zmp","name":"zmp_%i"%i,
                "params":{"planted_links":[from_foot, to_foot],"timestep":i}
            })
    request["constraints"].append(
        {
            "type":"pose",
            "name":"final_hand_pose",
            "params":{
                "xyz":list(hand_xyz),
                "wxyz":[0.5*np.sqrt(2),0,0,0.5*np.sqrt(2)],
                "link":hand_link,
                "pos_coeffs":[1,1,1],
                "rot_coeffs":[0.,0.,0.],
                "timestep":i
            }
        }        
    )

    return request

In [6]:
def check_traj_atlas(result, target, threshold = 0.01):
    traj = result.GetTraj()
    #check for collision
    is_safe = traj_is_safe(traj, robot)
    #print res
    
    if is_safe is not True:
        print "There is a collision within the trajectory!"
        return False

    #check target for pose constraints
    robot.SetActiveDOFValues(traj[-1])
    xyz = robot.GetLink('r_hand').GetTransform()[0:3,3]
    if (np.linalg.norm(xyz - target) > threshold):
        print('Target is not reached!')
        return False

    print 'Optimization is success!'
    return True

In [7]:
def generate_valid_xyz(limits, box_size = 0.04):
    is_success = False

    while is_success is not True:
        x,y,z = generate_random_xyz(limits)
        #create a box and check for collision
        is_col = check_col_with_box(env,x,y,z, box_size = box_size)
        if is_col:
            #print('There is collision!')
            continue
        else:
            #print('No collision!')
            is_success = True
    
    return x,y,z

def xyzQuatFromMatrix(T):
    wxyz_xyz = openravepy.poseFromMatrix(T)
    return wxyz_xyz[4:7], wxyz_xyz[0:4]

### Build / Retrieve Dataset

In [8]:
import pdb

In [9]:
def build_database(x_init, y_init, num_traj, limits, n_steps = 30,predictor=None):
    num_plan = 0
    comp_times = []
    costs = []
    tic = time.time()
    
    while len(x_init) < num_traj:
        print('{}th Planning, {} trajectories in database'.format(num_plan,len(x_init)))
        #Get a random pose as target
        init_joint = standing_posture.copy()
        robot.SetActiveDOFValues(init_joint)
        xyz_target = generate_valid_xyz(limits)
        robot.SetActiveDOFValues(init_joint)

        
        request_standard = press_button_request(robot, xyz_target, "r_hand", ["l_foot","r_foot"],n_steps)
        duration, result = run_opt(request_standard, env)
   
        #Check traj result
        traj = result.GetTraj()
        is_safe=  check_traj_atlas(result, xyz_target)
        #pdb.set_trace()
        if is_safe:
            print 'Planning is successfull!'
            x_init.append(xyz_target)
            y_init.append(traj.flatten())
            comp_times.append(duration)
            costs.append(result.GetCosts()[0][1])
            plot_traj(env,traj)
        else:
            print('Fail to find good solution!') 
            #plot_traj(env,traj)

        num_plan += 1
        
        
    #Store the result
    toc = time.time()
    total_time = toc-tic
    success_rate = num_traj*1.0/num_plan
    x_init = np.vstack(x_init)
    y_init = np.vstack(y_init)
    
    
    data = dict()
    data['x'] = x_init
    data['y'] = y_init
    data['total_time'] = total_time
    data['success_rate'] = success_rate
    data['comp_times'] = comp_times
    data['costs'] = costs
    return data

In [10]:
to_build_database = False
to_augment_database = False
num_traj = 500
n_steps = 15
limits = [[0.3, 0.7],[-0.5, 0.5],[0.6,1.4]]
#limits = [[0.3, 0.7],[-0.5, 0.5],[0.4,0.8]]

if to_build_database:
    if to_augment_database:
        #load data
        data = pickle.load( open(FILENAME + 'data.pkl', 'rb'))
        x_init = list(data['x'])
        y_init = list(data['y'])
        num_traj += len(x_init)     
    else:
        x_init,y_init = [],[]
    tic = time.time()
    data = build_database(x_init, y_init, num_traj,limits, n_steps = n_steps)
    toc = time.time()
    print('Planning time is {}'.format(toc-tic))
    x_init = data['x']
    y_init = data['y']
    pickle.dump(data,open(FILENAME + 'data.pkl', 'wb') )
    print('Success_rate : {}, average costs:{}'.format(data['success_rate'], np.mean(data['costs'])))
else:
    #load data
    data = pickle.load( open(FILENAME + 'data.pkl', 'rb'))
    x_init = data['x']
    y_init = data['y']
    num_traj = len(x_init)

In [11]:
x_init.shape

(1000, 3)

### Check and Plot the Trajectories

#### Selecting indexes 

In [12]:
indexes = []

In [13]:
indexes = [195,512,722,605]

In [20]:
robot.SetActiveDOFValues(traj[0])

In [21]:
time.sleep(4)
for index in indexes:
    pose = x_init[index]
    traj = y_init[index]
    traj = traj.reshape(-1,dof)
    print index
    ball = add_ball(env,pose[0],pose[1],pose[2],0.04)
    plot_traj(env,traj,0.1)
    time.sleep(0.5)
    env.RemoveKinBody(ball)
    

195
512
722
605


In [29]:
from planning_util import *

In [ ]:
box.InitFromSpheres()

In [23]:
env.RemoveKinBody(box)

True

env.Add#### Plotting properly 

In [15]:

'''
import gtk.gdk
def save_screenshot(x,y,w,h,file_name):
    window = gtk.gdk.get_default_root_window()
    sz = window.get_size()
    print "The size of the window is %d x %d" % sz
    pb = gtk.gdk.Pixbuf(gtk.gdk.COLORSPACE_RGB,False,8,sz[0],sz[1])
    pb = pb.get_from_drawable(window,window.get_colormap(),0,0,0,0,sz[0],sz[1])
    pb = pb.subpixbuf(int(x),int(y),int(w),int(h)) 
    if (pb != None):
        pb.save(file_name,"png")
        print "Screenshot saved."
    else:
        print "Unable to get the screenshot."'''
        
import pyscreenshot as ImageGrab
def save_screenshot(x,y,w,h,file_name, to_show=False):
    # part of the screen
    im=ImageGrab.grab(bbox=(x,y,w,h))
    if to_show:
        im.show()
    # save to file
    im.save(file_name)
    return im

In [17]:
viewer = trajoptpy.GetViewer(env)

In [ ]:
animate_traj(viewer,robot,traj)

In [ ]:
def animate_traj(viewer, robot, traj):
    for (i,row) in enumerate(traj):
        print "step",i
        robot.SetActiveDOFValues(row)
        viewer.Idle()
        save_screenshot(0,0,800,800,'fig'+str(i)+'.png')

In [ ]:
from memm

### Reduce the data sample

In [12]:
N_REDUCED = 1000
x_init = x_init[0:N_REDUCED]
y_init = y_init[0:N_REDUCED]

### Apply Dimensionality Reduction to the data

In [13]:
from sklearn.decomposition import PCA
y_pca = PCA(n_components = 50)
y_pca.fit(y_init)
y_init_reduced = y_pca.transform(y_init)

### Build Function Approximator

In [14]:
from IPython.display import clear_output

In [15]:
dim_input = len(x_init[0])

In [16]:
is_load_regressor = False

nn = NN_Regressor()
gpy = GPy_Regressor(dim_input = dim_input)
gpy_pca = GPy_Regressor(dim_input = dim_input, is_transform='PCA')
dpglm = DP_GLM_Regressor()
dpglm_pca = DP_GLM_Regressor(is_transform='PCA')

FILENAME_REDUCED = FILENAME+ str(N_REDUCED) + '_'

if is_load_regressor:
    nn.load_from_file(FILENAME_REDUCED + 'nn')
    gpy.load_from_file(FILENAME_REDUCED + 'gpy')
    gpy_pca.load_from_file(FILENAME_REDUCED + 'gpy_pca')
    dpglm.load_from_file(FILENAME_REDUCED + 'dpglm')
    dpglm_pca.load_from_file(FILENAME_REDUCED + 'dpglm_pca')
else:
    print 'Planning for NN'
    nn.fit(x_init, y_init)
    nn.save_to_file(FILENAME_REDUCED + 'nn')

    print 'Planning for GPY'
    gpy.fit(x_init, y_init, num_restarts=10)
    gpy.save_to_file(FILENAME_REDUCED + 'gpy')
    
    print 'Planning for GPY PCA'
    gpy_pca.fit(x_init, y_init_reduced)
    gpy_pca.save_to_file(FILENAME_REDUCED + 'gpy_pca')

    
    print 'Planning for DPGLM'
    dpglm.fit(x_init,y_init, n_components=30, weight_type = 'dirichlet_distribution')
    dpglm.save_to_file(FILENAME_REDUCED + 'dpglm')

    print 'Planning for DPGLM PCA'
    dpglm_pca.fit(x_init,y_init_reduced, n_components=30, weight_type = 'dirichlet_distribution')
    dpglm_pca.save_to_file(FILENAME_REDUCED + 'dpglm_pca')
    
clear_output()

### Evaluate the function approximators 

In [38]:
y_pca2 = PCA(n_components = 30)
y_pca2.fit(y_train)
y_train_reduced2 = y_pca2.transform(y_train)
dpglm_pca5 = DP_GLM_Regressor(is_transform='PCA')
dpglm_pca5.fit(x_train, y_train_reduced2, n_components = 30)
dpglm_pca5.pca = y_pca2


y_pca3 = PCA(n_components = 40)
y_pca3.fit(y_train)
y_train_reduced3 = y_pca3.transform(y_train)

dpglm_pca6 = DP_GLM_Regressor(is_transform='PCA')
dpglm_pca6.fit(x_train, y_train_reduced3, n_components = 30)
dpglm_pca6.pca = y_pca3



### Testing the warmstart

In [17]:
#creating the test cases
create_test = False
if create_test:
    x_test = []
    target_joints = []
    num_test = 300
    for i in range(num_test):
        init_joint = standing_posture.copy()
        robot.SetActiveDOFValues(init_joint)
        xyz_target = generate_valid_xyz(limits,box_size = 0.08)
        robot.SetActiveDOFValues(init_joint)
        cur_x = xyz_target
        x_test.append(cur_x)

    x_test = np.vstack(x_test)
    data_test = dict()
    data_test['x_test'] = x_test
    test_file = open(FILENAME + 'data_test.npy', 'wb')
    pickle.dump( data_test,test_file)
    test_file.close()
else:
    #load data
    test_file = open(FILENAME + 'data_test.npy', 'rb')
    data_test = pickle.load(test_file)
    x_test = data_test['x_test']
    num_test = len(x_test)
    init_joint = standing_posture.copy()

In [18]:
def set_init(request, init_type, init_values):
    request["init_info"]["type"] = "given_traj"

    if init_type == 'straight':
        request['init_info']['endpoint'] = init_values.tolist()
    elif init_type == 'given_traj':
        request["init_info"]["data"] = [row.tolist() for row in init_values]
    else:
        print('Initialization {} is not defined'.format(init_type))
        return None
    return request

In [19]:
from multiprocessing import Lock
from joblib import Parallel, delayed
import multiprocessing as mp

mutex = Lock()
   
def run_opt_special(request):
    robot.SetActiveDOFValues(init_joint)
    s = json.dumps(request) 
    prob = trajoptpy.ConstructProblem(s, env)
    tic = time.time()
    try:
        result = trajoptpy.OptimizeProblem(prob) # do optimization
    except:
        print 'Fault in optimization'
        return False, None, None, None
    toc = time.time()
    print("Optimization is completed in {} s!".format(toc-tic))
    duration = toc-tic
    
    #check the result
    try:
        success = check_traj_atlas(result, xyz_target)    
        print("Finish checking the trajectory1!")
        result_new = dict()
        result_new['traj'] = result.GetTraj()
        result_new['costs'] = result.GetCosts()[0][1]
        print("Finish checking the trajectory2!")
        cost = result.GetCosts()[0][1]
        result_new['status'] = result.GetStatus()
        print("Finish checking the trajectory3!")
        return success, duration, cost, result_new
    except:
        print 'Fault in checking traj!'
        return False, None, None, None
        
def function_callback(result):
    with mutex:
        print 'Obtain result:'
        print result[0]
        global count_proc
        global num_proc
        count_proc+= 1
        global ensemble_result
        ensemble_result = result
        if result[0]:
            print 'Terminating the mp'
            pool.terminate()
            return
        if count_proc == num_proc:
            print 'All fails, terminating the mp'
            pool.terminate()

In [22]:
method_names = ['STD    ',  'NN     ', 'GPY    ', 'BGMR   ', 'GPY_PCA   ', 'BGMR_PCA  ', 'ensemble']
methods = [nn, gpy, dpglm, gpy_pca, dpglm_pca]
#method_names = ['STD    ',  'NN     ', 'GPY    ']#, 'BGMR   ', 'GPY_PCA   ', 'BGMR_PCA  ']
#methods = [nn, gpy ]#dpglm, gpy_pca, dpglm_pca]

#method_names = ['STD    ',  'NN     ', 'GPY    ', 'BGMR   ', 'GPY_PCA   ', 'BGMR_PCA  ']
#methods = [nn, gpy, dpglm, gpy_pca, dpglm_pca]


results = dict()
result,success, duration = dict(),dict(),dict()
ensemble_result = None
for method in method_names:
    results[method] = dict()
    results[method]['costs'] = []
    results[method]['successes'] = []
    results[method]['comp_times'] = []
    results[method]['func_evals'] = []
    results[method]['qp_solves'] = []
    
num_test = 250
ik_times = []
quat = [1,0,0,0]
for it in range(num_test):
    print('{}th Planning'.format(it))
    
    #setting up the problem case
    #index = np.random.randint(0,len(x_test))
    index = it
    x_cur = x_test[index:index+1,:]
    xyz_target = x_cur.flatten()
    robot.SetActiveDOFValues(init_joint)
    
    #without warmstart
    method_name = method_names[0]
    request_standard = press_button_request(robot, xyz_target, "r_hand", ["l_foot","r_foot"],n_steps)
    duration[method_name], result[method_name] = run_opt(request_standard, env)
    success[method_name] = check_traj_atlas(result[method_name],xyz_target)
    
    
    #Other warmstart methods
    request_trajs = []
    for i,method in enumerate(methods):
        method_name = method_names[i+1]
        if isinstance(method, Straight_Regressor):
            traj,cov = method.predict(init_joint0, target_joint)
        elif isinstance(method, DP_GLM_Regressor):
            traj,cov = method.predict(x_cur, return_gmm = True)
        else:
            traj,cov = method.predict(x_cur)
            
        if method.is_transform == 'PCA':
            traj = y_pca.inverse_transform(traj)
        traj = traj.reshape(-1,dof)
        traj[0,:] = init_joint
        robot.SetActiveDOFValues(init_joint)
        request_traj = press_button_request(robot, xyz_target, "r_hand", ["l_foot","r_foot"],n_steps)
        request_traj = set_init(request_traj, 'given_traj', traj)
        duration[method_name], result[method_name] = run_opt(request_traj, env)    
        success[method_name] = check_traj_atlas(result[method_name],xyz_target)
        request_trajs.append(request_traj)
    


    #ensemble methods   
    num_proc = len(request_trajs)
    pool = mp.Pool(processes = num_proc)
    count_proc = 0
    tic = time.time()
    for request in request_trajs:
        pool.apply_async(run_opt_special, args = (request, ), callback = function_callback)
    time.sleep(15)
    pool.terminate()
    pool.close()
    pool.join()
    toc = time.time()
    print 'Ensemble takes ' + str(toc-tic)
    results['ensemble']['costs'].append(ensemble_result[2])
    results['ensemble']['successes'].append(ensemble_result[0])      
    results['ensemble']['comp_times'].append(toc-tic)
    
    
    
    #Record the result
    for method_name in method_names[:-1]:
        results[method_name]['costs'].append(result[method_name].GetCosts()[0][1])
        results[method_name]['func_evals'].append(result[method_name].GetNumFuncEval()[0])
        results[method_name]['qp_solves'].append(result[method_name].GetNumQPSolve()[0])
        results[method_name]['successes'].append(success[method_name])      
        results[method_name]['comp_times'].append(duration[method_name])
        
    print_result(results,method_names)

0th Planning
Optimization is completed in 5.41389894485 s!
There is a collision within the trajectory!


No handlers could be found for logger "openravepy"


Optimization is completed in 5.9282989502 s!
There is a collision within the trajectory!
Optimization is completed in 10.9944610596 s!
There is a collision within the trajectory!
Optimization is completed in 2.77910804749 s!
There is a collision within the trajectory!
Optimization is completed in 10.4383089542 s!
There is a collision within the trajectory!
Optimization is completed in 8.80462002754 s!
There is a collision within the trajectory!
Optimization is completed in 3.57158708572 s!
There is a collision within the trajectory!
Finish checking the trajectory1!
Finish checking the trajectory2!
Finish checking the trajectory3!
Obtain result:
False
Optimization is completed in 7.39181804657 s!
There is a collision within the trajectory!
Finish checking the trajectory1!
Finish checking the trajectory2!
Finish checking the trajectory3!
Obtain result:
False
Optimization is completed in 10.2542638779 s!
There is a collision within the trajectory!
Finish checking the trajectory1!
Finish c

 planning_util.py:164: RuntimeWarning:invalid value encountered in double_scalars
 /usr/lib/python2.7/dist-packages/numpy/core/_methods.py:82: RuntimeWarning:Degrees of freedom <= 0 for slice
 /usr/lib/python2.7/dist-packages/numpy/core/_methods.py:94: RuntimeWarning:invalid value encountered in true_divide
 /usr/lib/python2.7/dist-packages/numpy/core/_methods.py:116: RuntimeWarning:invalid value encountered in double_scalars
 planning_util.py:167: RuntimeWarning:invalid value encountered in double_scalars


Optimization is completed in 10.2513689995 s!
There is a collision within the trajectory!
Optimization is completed in 1.11778497696 s!
There is a collision within the trajectory!
Optimization is completed in 1.42299818993 s!
There is a collision within the trajectory!
Optimization is completed in 10.43562603 s!
There is a collision within the trajectory!
Optimization is completed in 7.6904399395 s!
There is a collision within the trajectory!
Optimization is completed in 7.72427797318 s!
There is a collision within the trajectory!
Optimization is completed in 1.36502504349 s!
There is a collision within the trajectory!
Finish checking the trajectory1!
Finish checking the trajectory2!
Finish checking the trajectory3!
Optimization is completed in 1.65530586243 s!
Obtain result:
False
There is a collision within the trajectory!
Finish checking the trajectory1!
Finish checking the trajectory2!
Finish checking the trajectory3!
Obtain result:
False
Optimization is completed in 12.6982250214 

### Printing Results

In [24]:
print_result(results,method_names)

 Method 	| Success Rate 	| Conv. Time 	| Traj. Cost 	| Func. Evals 	| QP Solves
STD     	& 50.800 	& 6.31$\pm$3.90 	& 0.12$\pm$0.07 \\
NN      	& 58.800 	& 1.48$\pm$1.39 	& 0.11$\pm$0.06 \\
GPY     	& 54.400 	& 1.29$\pm$1.09 	& 0.10$\pm$0.05 \\
BGMR    	& 56.400 	& 1.32$\pm$1.57 	& 0.10$\pm$0.05 \\
GPY_PCA    	& 60.000 	& 1.54$\pm$1.46 	& 0.11$\pm$0.05 \\
BGMR_PCA   	& 58.000 	& 1.36$\pm$1.16 	& 0.11$\pm$0.06 \\
ensemble 	& 71.200 	& 1.46$\pm$1.40 	& 0.12$\pm$0.06 \\


In [ ]:
print(' Method \t| Success Rate \t| Conv. Time \t| Traj. Cost \t| Func. Evals \t| QP Solves')
for method in method_names:
    successes = np.array(results[method]['successes'])
    success = np.count_nonzero(successes)
    
    comp_times = np.array(results[method]['comp_times'])[successes]
    costs = np.array(results[method]['costs'])[successes]
    #func_evals = np.array(results[method]['func_evals'])[successes]
    #qp_solves = np.array(results[method]['qp_solves']£)[successes]
    
    print('{0}: \t& {1:.3f} \t& {2:.2f} \t& {3:.3f}  \\\\'.format(method, success*1.0/len(successes), np.sum(comp_times)/success, np.sum(costs)/success))
    

### Save Result

In [25]:
pickle.dump(results,open(FILENAME_REDUCED + 'result3.pkl', 'wb') )

### Load Result 

In [19]:
results = pickle.load(open(FILENAME_REDUCED + 'result2.pkl', 'rb'))

In [22]:
x_test_success = x_test[:50][np.array(results['ensemble']['successes'])]

x_test_fail = x_test[np.invert(results['ensemble']['successes'])]

 /home/teguh/.local/lib/python2.7/site-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning:boolean index did not match indexed array along dimension 0; dimension is 300 but corresponding boolean dimension is 50


In [23]:
from planning_util import add_box

In [56]:
boxes = []
for x in x_test_fail:
    box = add_box(env, x[0], x[1],x[2])
    box.GetLink('base').GetGeometries()[0].SetDiffuseColor([1,0,0])
    boxes.append(box)

In [57]:
for x in x_test_success:
    box = add_box(env, x[0], x[1],x[2])
    box.GetLink('base').GetGeometries()[0].SetDiffuseColor([0,0,1])
    boxes.append(box)

In [29]:
box = boxes[0]

In [42]:
l = box.GetLink('base')

In [50]:
g = l.GetGeometries()[0]

In [51]:
g.SetDiffuseColor([1,0,0])

In [52]:
for box in boxes:
    env.RemoveKinBody(box)

### Printing Results Tied to Standard Success

In [51]:
print(' Method \t| Success Rate \t| Conv. Time \t| Traj. Cost \t| Func. Evals \t| QP Solves')
standard_successes = np.array(results['STD    ']['successes'])

for method in method_names:
    successes = np.array(results[method]['successes'])[standard_successes]
    success = np.count_nonzero(successes)
    comp_times = np.array(results[method]['comp_times'])[standard_successes][successes]
    costs = np.array(results[method]['costs'])[standard_successes][successes]
    func_evals = np.array(results[method]['func_evals'])[standard_successes][successes]
    qp_solves = np.array(results[method]['qp_solves'])[standard_successes][successes]
    
    print('{0}: \t {1:.3f} \t {2:.2f} \t {3:.3f} \t {4:.3f} \t {5:.3f}'.format(method, success*1.0/len(successes), np.sum(comp_times)/success, np.sum(costs)/success, 1.0*np.sum(func_evals)/success, 1.0*np.sum(qp_solves)/success ))
    

 Method 	| Success Rate 	| Conv. Time 	| Traj. Cost 	| Func. Evals 	| QP Solves
STD    : 	 1.000 	 8.50 	 0.142 	 80.200 	 79.200
NN     : 	 0.850 	 1.86 	 0.125 	 19.353 	 18.353
GPY    : 	 0.900 	 1.10 	 0.124 	 12.000 	 11.000
BGMR   : 	 1.000 	 1.47 	 0.127 	 15.250 	 14.250
GPY_PCA   : 	 1.000 	 1.26 	 0.126 	 13.450 	 12.450
BGMR_PCA  : 	 1.000 	 1.54 	 0.125 	 16.600 	 15.600


 /home/teguh/.local/lib/python2.7/site-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning:boolean index did not match indexed array along dimension 0; dimension is 0 but corresponding boolean dimension is 50


IndexError: index 1 is out of bounds for axis 1 with size 0